In [15]:
import pandas as pd
import numpy as np
import torch
from transformers import pipeline
import sentence_transformers

model = pipeline(model="seara/rubert-tiny2-russian-sentiment")
model_cossim = sentence_transformers.SentenceTransformer('inkoziev/sbert_synonymy')

In [8]:
df = pd.read_csv('/kaggle/input/extended-cra/dataset.csv')
df.head(2)

,Unnamed: 0,pr_txt,Категория,Уровень рейтинга,preprocessed_text,target,scaled_target
0,0,Повышение кредитного рейтинга Акционерного общ...,A,A,повышение кредитный акционерный общество ураль...,3,0.65
1,1,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,подтвердить кредитный энергоконцепт уровень мо...,-3,0.35


In [9]:
def get_sentiment(text, model, token_len=5, step=5, CRS='AA', sort_by = ['positive', 'neutral']):
    df_score = pd.DataFrame(columns=['batch', 'positive', 'neutral', 'negative'])
    
    #get all tokens with (token_len)-batch size and step window-size
    splitted = text.split()
    tokens = [' '.join(splitted[i:i+token_len]) for i in range(0,len(splitted), step)]
    
    #extract windows where positive_score > n or neutral_score > k while positive_score > negative_score
    tkn_dict = {}
    for token in tokens:
        sentiment_dict = model(token, top_k=3)
        answ = {}
        for i, d in enumerate(sentiment_dict):
            answ[d['label']] = d['score'] 
        
        #in different cases we look for either positive or negative sentiment
        if CRS[0] == 'A':
            pos_score=0.6; neut_score=0.85; neg_score=0.2;
            if (answ['neutral'] >= neut_score and (answ['negative'] <= 2*answ['positive'])) \
            or answ['positive'] >= pos_score:
                tkn_dict[token] = answ
                sort_by = ['positive', 'neutral']
        elif len(CRS) > 1 and CRS[0] == 'B':
            neut_score=0.85
            if answ['neutral'] >= neut_score:
                tkn_dict[token] = answ
        else:
            pos_score=0.2; neut_score=0.85; neg_score=0.6;
            if (answ['neutral'] >= neut_score and (2*answ['negative'] >= answ['positive'])) \
            or answ['negative'] >= neg_score:
                tkn_dict[token] = answ
                sort_by = ['negative', 'neutral']
    
        df_score.loc[len(df_score)] = [token, answ['positive'], answ['neutral'], answ['negative']]
        df_score = df_score.sort_values(by=sort_by, ascending=False)

    return tkn_dict, df_score

In [44]:
#top_k - top k of tokens sorted by sentiment
#top_n - top n matched phrases sorted by cos_sim
def locate(raw_text, tokens, top_k=20, top_n=20):
    df_cossim = pd.DataFrame(columns=['cossim', 'token', 'raw_text'])
    top_tokens = tokens[1:top_k+1]
    sentences = raw_text.split('.')
    embeddings = model_cossim.encode(sentences)
    
    raw_text_located = []
    for token in top_tokens:
        v1 = model_cossim.encode(token)
        max_cossim = 0
        max_match = ''
        for i2 in range(len(embeddings)):
            s = sentence_transformers.util.cos_sim(a=v1, b=embeddings[i2]).item()
            if s > max_cossim:
                max_cossim = s; max_match = sentences[i2]
        raw_text_located.append({f'{token}': max_match})
        
        df_cossim.loc[len(df_cossim)] = [s, token, max_match]
        
    df_cossim = df_cossim.sort_values(by=['cossim'], ascending=False)[1:top_n+1]
    return raw_text_located, df_cossim

In [36]:
%%time
for i in [7, 29, 90, 123, 764]:
    crs = df['Категория'][i]
    tkn_dict, df_score = get_sentiment(df.preprocessed_text.values[i], model, CRS=crs, token_len=7, step=7)
    located_dict, located_df = locate(df.pr_txt.values[i], df_score.batch.to_list())
    print(crs)
    
    for j in located_df.index.to_list()[:5]:
        print(located_df.token[j], located_df.raw_text[j], sep='\n')
        print('\n')

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

A
последний месяц отчетный дата далее отчетный период
 Представленная информация актуальна на момент публикации


год дата настоящий прессрелиз присвоение кредитный использоваться
Кредитный рейтинг присвоен в рамках заключенного договора ООО «Велесстрой» приняло участие в присвоении рейтинга


привлечение банковский финансирование исполнение новый контракт качественный
факты или рекомендации покупать, держать или продавать определенные ценные бумаги или активы, принимать инвестиционные решения


рейтинговый агентство пересмотреть кредитный финансовый обновить методология
 Рейтинговое агентство «Эксперт РА» пересмотрело кредитный рейтинг не- финансовой компании "Велесстрой" по обновленной методологии присвоен рейтинг на уровне <рейтинг>, прогноз по рейтингу - стабильный


высокий зависимость объем договорный база наличие отдельный
 Агентство отмечает, что динамика изменения показателя исторически сильно зависит от специфики контрактов, выполняемых компанией в каждый конкретный период




In [47]:
%%time
tkn_dict, df_score = get_sentiment(df.preprocessed_text.values[i], model, CRS='BBB', token_len=7, step=7)
located_dict, located_df = locate(df.pr_txt.values[i], df_score.batch.to_list(), top_n=7)
key_phrases_raw = located_df.raw_text.to_list()

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 3.14 s, sys: 54.3 ms, total: 3.2 s
Wall time: 1.83 s


In [48]:
key_phrases_raw

['Кредитный рейтинг присвоен в рамках заключенного договора ООО «Велесстрой» приняло участие в присвоении рейтинга',
 'факты или рекомендации покупать, держать или продавать определенные ценные бумаги или активы, принимать инвестиционные решения',
 ' Предыдущий рейтинговый пресс-релиз по данному объекту рейтингования был опубликован 06',
 ' Рейтинговое агентство «Эксперт РА» пересмотрело кредитный рейтинг не- финансовой компании "Велесстрой" по обновленной методологии присвоен рейтинг на уровне <рейтинг>, прогноз по рейтингу - стабильный',
 ' Агентство отмечает, что динамика изменения показателя исторически сильно зависит от специфики контрактов, выполняемых компанией в каждый конкретный период',
 ' Показатели рентабельности свободного денежного потока имеют аналогичные оценки',
 'Рыночное положение ООО «Велесстрой» оценивается агентством как высокое']